In [ ]:
#! pip install -U climetlab --quiet
#! pip install -U climetlab_s2s_ai_challenge --quiet

In [ ]:
import climetlab as cml 

In [3]:
import climetlab_s2s_ai_challenge
print(f'Climetlab version : {cml.__version__}')
print(f'Climetlab-s2s-ai-challenge plugin version : {climetlab_s2s_ai_challenge.__version__}')

Climetlab version : 0.5.6
Climetlab-s2s-ai-challenge plugin version : 0.4.16


In [12]:
import os
# When running in continous integration in github,
# append "-dev" to the datasets name to download only a fragment of data
# Warning : do not use the "-dev" datasets for training ML models.
if os.environ.get('GITHUB_ACTIONS'):
    is_test = '-dev'
else:
    is_test = ''

In [13]:
# https://storage.ecmwf.europeanweather.cloud/s2s-ai-challenge/data/observations{is_test}/{param}-{freq}-since-2002.nc

# Observations data for training

Climetlib provides the observation datasets. They can be used as a xarray.Dataset :

In [14]:
cmlds = cml.load_dataset("s2s-ai-challenge-observations"+is_test,
                         freq='daily',
                         parameter='t2m'                         
                       )
cmlds.to_xarray()

<xarray.Dataset>
Dimensions:    (latitude: 121, longitude: 240, time: 7067)
Coordinates:
  * time       (time) datetime64[ns] 2001-12-24 2001-12-25 ... 2021-04-29
  * latitude   (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude  (longitude) float64 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
Data variables:
    t2m        (time, latitude, longitude) float32 ...

In [15]:
cmlds = cml.load_dataset("s2s-ai-challenge-observations"+is_test,
                         freq='weekly',
                         parameter='t2m'                         
                       )
cmlds.to_xarray()

<xarray.Dataset>
Dimensions:    (latitude: 121, longitude: 240, time: 1009)
Coordinates:
  * time       (time) datetime64[ns] 2002-01-03 2002-01-10 ... 2021-04-29
  * latitude   (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude  (longitude) float64 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
Data variables:
    t2m        (time, latitude, longitude) float32 ...